# boh roba

In [2]:
# inclusion
import sklearn.datasets
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg
import sklearn as sk

In [3]:
# utility functions

def load_iris():
    D,L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    return D, L

def vcol(v):
    return np.array(v).reshape(v.size, 1)

def vrow(v):
    return np.array(v).reshape(1, v.size)

In [4]:
#split
def split(D, L, seed=0):
    nTrain = int(D.shape[1]*2.0/3.0)
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1])
    idxTrain = idx[0:nTrain]
    idxTest = idx[nTrain:]
    DTR = D[:, idxTrain]
    DTE = D[:, idxTest]
    LTR = L[idxTrain]
    LTE = L[idxTest]
    return (DTR, LTR), (DTE, LTE)

In [5]:
D, L = load_iris()
(DTR, LTR), (DTE, LTE) = split(D, L)

In [17]:
# Max Likelihood parameters:
Nc = np.unique(LTR).size
mu = np.zeros((4,Nc))
for c in range(Nc):
    DTR_c = DTR[:, LTR == c]
    mu = vcol(DTR_c.mean(axis=1))
    C = ((DTR_c-mu)@(DTR_c-mu).T)/DTR_c.shape[1]
    print(f"mu_{c}: {mu}\nC_{c}:{C}\n")

mu_0: [[4.96129032]
 [3.42903226]
 [1.46451613]
 [0.2483871 ]]
C_0:[[0.13140479 0.11370447 0.02862643 0.01187305]
 [0.11370447 0.16270552 0.01844953 0.01117586]
 [0.02862643 0.01844953 0.03583767 0.00526535]
 [0.01187305 0.01117586 0.00526535 0.0108845 ]]

mu_1: [[5.91212121]
 [2.78484848]
 [4.27272727]
 [1.33939394]]
C_1:[[0.26470156 0.09169881 0.18366391 0.05134068]
 [0.09169881 0.10613407 0.08898072 0.04211203]
 [0.18366391 0.08898072 0.21955923 0.06289256]
 [0.05134068 0.04211203 0.06289256 0.03208448]]

mu_2: [[6.45555556]
 [2.92777778]
 [5.41944444]
 [1.98888889]]
C_2:[[0.30080247 0.08262346 0.18614198 0.04311728]
 [0.08262346 0.08533951 0.06279321 0.05114198]
 [0.18614198 0.06279321 0.18434414 0.04188272]
 [0.04311728 0.05114198 0.04188272 0.0804321 ]]

